# PSF Photometry Basics with Photutils

This notebook provides an basic overview of PSF photometry using [Photutils](https://photutils.readthedocs.io/en/stable/).

**Use case:** Create a simulated image of stars, PSF photometry.<br>
**Data:** This notebook creates a JWST/NIRCam F200W NRCA1 simulated image of stars.<br>
**Tools:**  photutils.<br>
**Instrument:** NIRCam.<br>
**Documentation:** This notebook is part of a STScI's larger [post-pipeline Data Analysis Tools Ecosystem](https://jwst-docs.stsci.edu/jwst-post-pipeline-data-analysis).<br>

# Imports

In [ ]:
import os

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import requests
from astropy.table import QTable
from astropy.visualization import simple_norm
from photutils.aperture import CircularAperture
from photutils.datasets import make_noise_image
from photutils.detection import IRAFStarFinder
from photutils.psf import (GriddedPSFModel, PSFPhotometry, SourceGrouper,
                           make_psf_model_image)
from photutils.utils import make_random_cmap
from tweakwcs import XYXYMatch

# Change some default plotting parameters
mpl.rcParams['image.origin'] = 'lower'
mpl.rcParams['image.interpolation'] = 'nearest'

# Point Spread Function Photometry with Photutils

The Photutils PSF photometry module provides modular tools that allow users to fully customize the photometry procedure (e.g., by using different source detection algorithms, local background estimators, source groupers, and PSF models). Photutils provides implementations for each subtask involved in the photometry process. However, users can also incorporate their own custom implementations for any of the tasks, if desired. This modularity is achieved through an object-oriented programming approach, providing a more convenient user experience.

Photutils provides two top-level classes to perform PSF photometry, `PSFPhotometry` and `IterativelyPSFPhotometry`. In this notebook, we will cover the basics of the `PSFPhotometry` class.

# PSF Models

PSF photometry fundamentally involves fitting models to data. As such, the PSF model is a critical component of PSF photometry. For accurate results in both photometry and astrometry, the PSF model should closely represent the actual data. The PSF model can be a simple analytic function, such as a 2D Gaussian or Moffat profile, or it can be a more complex model derived from a 2D PSF image (e.g., an effective PSF, or ePSF). The PSF model can also account for variations in the PSF across the detector (e.g., due to optical aberrations).

Photutils provides the following analytic PSF Models based on the [Astropy Modeling and Fitting framework](https://docs.astropy.org/en/latest/modeling/index.html):

* [GaussianPSF](https://photutils.readthedocs.io/en/latest/api/photutils.psf.GaussianPSF.html#photutils.psf.GaussianPSF)
* [CircularGaussianPSF](https://photutils.readthedocs.io/en/latest/api/photutils.psf.CircularGaussianPSF.html#photutils.psf.CircularGaussianPSF)
* [MoffatPSF](https://photutils.readthedocs.io/en/latest/api/photutils.psf.MoffatPSF.html#photutils.psf.MoffatPSF)
* [AiryDiskPSF](https://photutils.readthedocs.io/en/latest/api/photutils.psf.AiryDiskPSF.html#photutils.psf.AiryDiskPSF)
* [GaussianPRF](https://photutils.readthedocs.io/en/latest/api/photutils.psf.GaussianPRF.html#photutils.psf.GaussianPRF)
* [CircularGaussianPRF](https://photutils.readthedocs.io/en/latest/api/photutils.psf.CircularGaussianPRF.html#photutils.psf.CircularGaussianPRF)

It also includes image-based PSF models:

* [ImagePSF](https://photutils.readthedocs.io/en/latest/api/photutils.psf.ImagePSF.html#photutils.psf.ImagePSF)
* [GriddedPSFModel](https://photutils.readthedocs.io/en/latest/api/photutils.psf.ImagePSF.html#photutils.psf.ImagePSF)

## Create a Photutils `GriddedPSFModel`

A gridded PSF model is a grid of position-dependent ePSFs that takes into account the PSF varying across the detector.

Let's use a position-dependent gridded PSF model for JWST/NIRCam F200W in detector NRCA1.  

This gridded PSF model was calculated using [STPSF](https://stpsf.readthedocs.io/en/latest/), a Python package that computes simulated PSFs for both JWST and Roman.
 
The next cell shows how to use STPSF to generate the gridded PSF model. However, for this notebook we'll simply download a pre-calculated gridded PSF model (see the following cell).

In [ ]:
# How to calculate the GriddedPSFModel file that we load in the next cell
# nrc = stpsf.NIRCam()
# nrc.filter = 'F200W'
# nrc.detector = 'NRCA1'
# psf_model = nrc.psf_grid(num_psfs=16, all_detectors=False, verbose=True, save=True)

In [ ]:
filename = "nircam_nrca1_f200w_fovp101_samp4_npsf16.fits"
baseurl = "https://data.science.stsci.edu/redirect/JWST/jwst-data_analysis_tools/stpsf_grid/"
url = os.path.join(baseurl, filename)
file_path = os.path.join('.', filename)

if not os.path.exists(file_path):
    response = requests.get(url)
    with open(file_path, 'wb') as file:
        file.write(response.content)
        print(f"File saved as: {file_path}")
else:
    print(f"File already exists: {file_path}")

In [ ]:
# load the STPSF GriddedPSF model from the downloaded FITS file
filename = 'nircam_nrca1_f200w_fovp101_samp4_npsf16.fits'
psf_model = GriddedPSFModel.read(filename)
psf_model

`psf_model` contains a 3D cube of PSFs.  There are 16 2D PSFs, each 404 x 404 pixels.

In [ ]:
psf_model.data.shape

The default oversampling is 4 along each axis.

In [ ]:
psf_model.oversampling

Let's plot the grid of ePSFs.

In [ ]:
fig = psf_model.plot_grid(vmax_scale=0.1, figsize=(9, 9))

In [ ]:
fig = psf_model.plot_grid(deltas=True, figsize=(9, 9))

# Let's use this PSF model to create an image of simulated stars

We'll use the `photutils.psf` [make_psf_model_image](https://photutils.readthedocs.io/en/latest/api/photutils.psf.make_psf_model_image.html) function.

### We'll create 500 stars in a 2048 x 2048 image (NIRCam F200W NRCA1)

In [ ]:
n_sources = 500
shape = (2048, 2048)
data, true_params = make_psf_model_image(shape, psf_model, n_sources,
                                         flux=(500, 20_000), min_separation=25,
                                         seed=0, progress_bar=True)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
norm = simple_norm(data, 'sqrt', percent=98)
axim = ax.imshow(data, norm=norm)

Now let's add some Gaussian noise (σ = 0.5) to the image.

In [ ]:
noise = make_noise_image(data.shape, mean=0, stddev=0.5, seed=0)
data += noise
error = np.sqrt(np.abs(data))
fig, ax = plt.subplots(figsize=(10, 10))
norm2 = simple_norm(data, 'sqrt', percent=99)
axim = ax.imshow(data, norm=norm2)

The `true_params` output contains an Astropy table containing the true (x, y, flux) of our artificial stars.

In [ ]:
true_params

# Finding Stars in an Image [(photutils.detection)](https://photutils.readthedocs.io/en/latest/user_guide/detection.html)

Let's use the [IRAFStarFinder class](
https://photutils.readthedocs.io/en/latest/api/photutils.detection.IRAFStarFinder.html#photutils.detection.IRAFStarFinder)
 to find the stars in the simulated image.

In [ ]:
finder = IRAFStarFinder(threshold=6.0, fwhm=3.0)
stars = finder(data)
stars

The star finder found all 500 stars in the image. Let's plot circles around the detected stars.

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
axim = ax.imshow(data, norm=norm2)
xypos = zip(stars['xcentroid'], stars['ycentroid'])
aper = CircularAperture(xypos, r=20)
patches = aper.plot(ax=ax, color='red')

# The `PSFPhotometry` class

In this example, we'll perform PSF photometry on our simulated image.

First, we create the `PSFPhotometry` class instance with a few parameters.

We must input a PSF model, which must be an Astropy `Fittable2DModel`. As described above, Photutils provides several PSF models, including a [`GriddedPSFModel`](https://photutils.readthedocs.io/en/latest/api/photutils.psf.GriddedPSFModel.html#photutils.psf.GriddedPSFModel) for spatially-varying PSFs.

We must also input the `fit_shape` parameter, which defines the region around the center of each detected star that is used for fitting the PSF model.

We must also provide initial guesses of the position and flux for each star in order to perform the model fitting. There are a few ways to accomplish that. In this example, we'll using the optional `finder` and `aperture_radius` keywords.  The `finder` will be used internally to detect the sources and calculate their initial (x, y) positions. The `aperture_radius` (in pixels) will be used internally to calculate the initial flux values for each source. For other options (e.g., inputting an `init_params` table), please see the [PSF Photometry docs](https://photutils.readthedocs.io/en/latest/user_guide/psf.html).

We set `progress_bar=True` to display an interactive progress bar during the PSF fitting.

In [ ]:
fit_shape = (5, 5)
finder = IRAFStarFinder(threshold=6.0, fwhm=3.0)
psfphot = PSFPhotometry(psf_model, fit_shape, finder=finder, aperture_radius=5, progress_bar=True)

To perform the PSF fitting, we call the `psfphot` object on the data and optional error array.  The result is an Astropy [QTable](https://docs.astropy.org/en/latest/table/index.html) with the fit results.

In [ ]:
phot = psfphot(data, error=error)
phot

The `*_init` columns contain the initial (x, y, flux) values.  The corresponding `*_fit` columns contain the results of the PSF fitting and the `*_err` columns contain the errors on each fit parameter.

We can now use the `make_model_image` method to create a PSF model image of the results.

In [ ]:
model_img = psfphot.make_model_image(data.shape)

fig, ax = plt.subplots(figsize=(10, 10))
axim = ax.imshow(model_img, norm=norm)

We can use the `make_residual_image` method to create a residual image.

In [ ]:
resid = psfphot.make_residual_image(data)

fig, ax = plt.subplots(figsize=(10, 10))
norm3 = simple_norm(data, 'sqrt', percent=95)
axim = ax.imshow(resid, norm=norm3)

Our residual image is just noise without any sources, which indicates excellent PSF model fits.

## Comparing results

Let's use our knowledge of the true (x, y) positions and fluxes to compare to our PSF fit results.

We first need to match the table catalogs.

In [ ]:
# convenience function to match (x, y) positions
def xymatch_catalogs(ref_params, params):
    refcat = QTable()
    refcat['TPx'] = ref_params['x_0']
    refcat['TPy'] = ref_params['y_0']
    fitcat = QTable()
    fitcat['TPx'] = params['x_fit']
    fitcat['TPy'] = params['y_fit']
    match = XYXYMatch(separation=1)
    ref_idx, fit_idx = match(refcat, fitcat)

    return ref_params[ref_idx], params[fit_idx]

In [ ]:
true_params, fit_params = xymatch_catalogs(true_params, phot)

In [ ]:
fig, ax = plt.subplots(ncols=3, figsize=(12, 4))
fig.suptitle('PSF Photometry Results')
ax[0].plot(true_params['x_0'], fit_params['x_fit'], '.')
ax[0].set_xlabel('True x')
ax[0].set_ylabel('Fit x')
ax[1].plot(true_params['y_0'], fit_params['y_fit'], '.')
ax[1].set_xlabel('True y')
ax[1].set_ylabel('Fit y')
ax[2].plot(true_params['flux'], fit_params['flux_fit'], '.')
ax[2].set_xlabel('True Flux')
ax[2].set_ylabel('Fit Flux')
plt.tight_layout()

In [ ]:
fig, ax = plt.subplots()
pdiff = (true_params['flux'] - fit_params['flux_fit']) / true_params['flux'] * 100.0
ax.set_title('Histogram of PSF Flux Differences')
ax.hist(pdiff, bins=50)
text = ax.set_xlabel('Percent Difference (between True and Fit)')

# Source Grouping

Source grouping is an optional feature that allows you to group close stars that should be fit simultaneously.

To turn it on, create a [SourceGrouper](https://photutils.readthedocs.io/en/latest/api/photutils.psf.SourceGrouper.html#photutils.psf.SourceGrouper) instance and input it via the grouper keyword. Here we’ll group stars that are within 20 pixels of each other:

In [ ]:
finder = IRAFStarFinder(threshold=6.0, fwhm=3.0)
stars = finder(data)
stars[0:5]

In [ ]:
min_separation = 35
grouper = SourceGrouper(min_separation)

In [ ]:
x = np.array(stars['xcentroid'])
y = np.array(stars['ycentroid'])
group_ids = grouper(x, y)

`group_ids` is an array with 500 elements (1 per input (x, y) position) with the group IDs.

In [ ]:
group_ids

The grouping algorithm separated the 500 stars into 428 distinct groups

In [ ]:
print(max(group_ids))

Let's visualize them. Close groups of stars with the same circle color are in the same group.

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
norm4 = simple_norm(data, 'sqrt', percent=99)
ax.imshow(data, norm=norm4, cmap='Greys_r')
cmap = make_random_cmap(ncolors=500, seed=123)
for i in np.arange(1, max(group_ids) + 1):
    mask = group_ids == i
    xypos = zip(x[mask], y[mask])
    ap = CircularAperture(xypos, r=20)
    ap.plot(color=cmap.colors[i], lw=2)

For example, the six stars outlined with light orange circles in the figure below are all in the same group.

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))
norm4 = simple_norm(data, 'sqrt', percent=99)
ax.imshow(data, norm=norm4, cmap='Greys_r')
cmap = make_random_cmap(ncolors=500, seed=123)
for i in np.arange(1, max(group_ids) + 1):
    mask = group_ids == i
    xypos = zip(x[mask], y[mask])
    ap = CircularAperture(xypos, r=20)
    ap.plot(color=cmap.colors[i], lw=2)
ax.set_xlim(1050, 1250)
ax.set_ylim(1300, 1500)

To perform grouped PSF photometry, we can simply input the `SourceGrouper` instance into the `grouper` keyword.

In [ ]:
fit_shape = (5, 5)
finder = IRAFStarFinder(threshold=6.0, fwhm=3.0)
min_separation = 35
grouper = SourceGrouper(min_separation)
psfphot = PSFPhotometry(psf_model, fit_shape, finder=finder, grouper=grouper, 
                        aperture_radius=5, progress_bar=True)
phot = psfphot(data, error=error)
phot

The `group_id` and `group_size` columns can be used to determine which stars were grouped together and how many stars were in each group.

# Fitting a single source

Instead of finding and fitting all stars in an image, we can instead fit only a few or a single star if desired.

Let's randomly select a single star from our image (circled in red in the image below).

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
axim = ax.imshow(data, norm=norm2)

x = 743
y = 1044
aper = CircularAperture((x, y), r=20)
patches = aper.plot(color='red')

In this example, we will not input the star finder (`finder = None`).  Instead, we'll input an Astropy table containing initial (x, y) position guesses for the stars (here a single star) that we want to fit.  We'll still input `aperture_radius` to estimate the initial flux.  However, we could also include the initial flux in the `init_params` table, in which case `aperture_radius` would not be needed.

In [ ]:
init_params = QTable()
init_params['x'] = [x]
init_params['y'] = [y]

fit_shape = (5, 5)
psfphot = PSFPhotometry(psf_model, fit_shape, finder=None, 
                        aperture_radius=5, progress_bar=True)
phot = psfphot(data, error=error, init_params=init_params)
phot

In [ ]:
resid = psfphot.make_residual_image(data)

fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(15, 5))
norm = simple_norm(data, 'sqrt', percent=99)
ax[0].imshow(data, norm=norm)
ax[1].imshow(data - resid, norm=norm)
im = ax[2].imshow(resid, norm=norm)
ax[0].set_title('Data')
aper.plot(ax=ax[0], color='red')
ax[1].set_title('Model')
aper.plot(ax=ax[1], color='red')
ax[2].set_title('Residual Image')
aper.plot(ax=ax[2], color='red')
plt.tight_layout()

# Further Reading

Please consult the [PSF Photometry documentation](https://photutils.readthedocs.io/en/stable/user_guide/psf.html) for additional features, including:

- Forced Photometry
- Fixed Model Parameters
- Bounded Model Parameters
- Iterative PSF Photometry
- Local Background Subtraction

## About this Notebook

**Author**: Larry Bradley, Branch Deputy, Data Analysis Tools Branch\
**Created**: 2025-07-16

<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/>